In [572]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from glob import glob
from scipy import misc
import os
from tqdm import tqdm
import pandas as pd
import colorsys
import hog_upd

%matplotlib inline

Лог регрессия, моя старая реалиазация для deep bayes (она выдавалась почти готовой, там нужно было буквально пару строчек написать)

In [872]:
# -*- coding: utf-8 -*-

import numpy as np
import scipy
import time
from collections import defaultdict
from scipy.optimize.linesearch import scalar_search_wolfe2

# Используйте scipy.special для вычисления численно неустойчивых функций
# https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special


class LineSearchTool(object):
    """
    Line search tool for adaptively tuning the step size of the algorithm.
    method : String containing 'Wolfe', 'Armijo' or 'Constant'
        Method of tuning step-size.
        Must be be one of the following strings:
            - 'Wolfe' -- enforce strong Wolfe conditions;
            - 'Armijo" -- adaptive Armijo rule;
            - 'Constant' -- constant step size.
    kwargs :
        Additional parameters of line_search method:
        If method == 'Wolfe':
            c1, c2 : Constants for strong Wolfe conditions
            alpha_0 : Starting point for the backtracking procedure
                to be used in Armijo method in case of failure of Wolfe method.
        If method == 'Armijo':
            c1 : Constant for Armijo rule
            alpha_0 : Starting point for the backtracking procedure.
        If method == 'Constant':
            c : The step size which is returned on every step.
    """
    def __init__(self, method='Wolfe', **kwargs):
        self._method = method
        if self._method == 'Wolfe':
            self.c1 = kwargs.get('c1', 1e-4)
            self.c2 = kwargs.get('c2', 0.9)
            self.alpha_0 = kwargs.get('alpha_0', 1.0)
        elif self._method == 'Armijo':
            self.c1 = kwargs.get('c1', 1e-4)
            self.alpha_0 = kwargs.get('alpha_0', 1.0)
        elif self._method == 'Constant':
            self.c = kwargs.get('c', 1.0)
        else:
            raise ValueError('Unknown method {}'.format(method))

    @classmethod
    def from_dict(cls, options):
        if type(options) != dict:
            raise TypeError('LineSearchTool initializer must be of type dict')
        return cls(**options)

    def to_dict(self):
        return self.__dict__

    def line_search(self, oracle, x_k, d_k, previous_alpha=None):
        alpha = None
        if self._method == 'Constant':
            alpha = self.c
        elif self._method == 'Wolfe':
            found = scalar_search_wolfe2(lambda al: oracle.func(x_k + al * d_k),
                                         lambda al: oracle.grad(x_k + al * d_k).T.dot(d_k),
                                         phi0=oracle.func(x_k),
                                         derphi0=oracle.grad(x_k).T.dot(d_k),
                                         c1=self.c1,
                                         c2=self.c2,
                                         amax=previous_alpha if previous_alpha is not None else
                                         self.alpha_0)
            alpha = found[0]
        if (self._method == 'Armijo') | (not alpha):
            alpha = previous_alpha if previous_alpha is not None else self.alpha_0
            phi_zero = oracle.func(x_k)
            phi_der_zero = oracle.grad(x_k).T.dot(d_k)
            while oracle.func(x_k + alpha * d_k) >= phi_zero + self.c1 * alpha * phi_der_zero:
                alpha = alpha * 0.5
        """
        Finds the step size alpha for a given starting point x_k
        and for a given search direction d_k that satisfies necessary
        conditions for phi(alpha) = oracle.func(x_k + alpha * d_k).
        Parameters
        ----------
        oracle : BaseSmoothOracle-descendant object
            Oracle with .func_directional() and .grad_directional() methods implemented for computing
            function values and its directional derivatives.
        x_k : np.array
            Starting point
        d_k : np.array
            Search direction
        previous_alpha : float or None
            Starting point to use instead of self.alpha_0 to keep the progress from
             previous steps. If None, self.alpha_0, is used as a starting point.
        Returns
        -------
        alpha : float or None if failure
            Chosen step size
        """
        return alpha


def get_line_search_tool(line_search_options=None):
    if line_search_options:
        if type(line_search_options) is LineSearchTool:
            return line_search_options
        else:
            return LineSearchTool.from_dict(line_search_options)
    else:
        return LineSearchTool()


def gradient_descent(oracle, x_0, tolerance=1e-5, max_iter=10000,
                     line_search_options=None, trace=False, display=False, method=None):
    """
    Gradien descent optimization method.
    Parameters
    ----------
    oracle : BaseSmoothOracle-descendant object
        Oracle with .func(), .grad() and .hess() methods implemented for computing
        function value, its gradient and Hessian respectively.
    x_0 : np.array
        Starting point for optimization algorithm
    tolerance : float
        Epsilon value for stopping criterion.
    max_iter : int
        Maximum number of iterations.
    line_search_options : dict, LineSearchTool or None
        Dictionary with line search options. See LineSearchTool class for details.
    trace : bool
        If True, the progress information is appended into history dictionary during training.
        Otherwise None is returned instead of history.
    display : bool
        If True, debug information is displayed during optimization.
        Printing format and is up to a student and is not checked in any way.
    Returns
    -------
    x_star : np.array
        The point found by the optimization procedure
    message : string
        "success" or the description of error:
            - 'iterations_exceeded': if after max_iter iterations of the method x_k still doesn't satisfy
                the stopping criterion.
            - 'computational_error': in case of getting Infinity or None value during the computations.
    history : dictionary of lists or None
        Dictionary containing the progress information or None if trace=False.
        Dictionary has to be organized as follows:
            - history['time'] : list of floats, containing time in seconds passed from the start of the method
            - history['func'] : list of function values f(x_k) on every step of the algorithm
            - history['grad_norm'] : list of values Euclidian norms ||g(x_k)|| of the gradient on every step of the algorithm
            - history['x'] : list of np.arrays, containing the trajectory of the algorithm. ONLY STORE IF x.size <= 2
    Example:
    --------
    >> oracle = QuadraticOracle(np.eye(5), np.arange(5))
    >> x_opt, message, history = gradient_descent(oracle, np.zeros(5), line_search_options={'method': 'Armijo', 'c1': 1e-4})
    >> print('Found optimal point: {}'.format(x_opt))
       Found optimal point: [ 0.  1.  2.  3.  4.]
    """
    history = defaultdict(list) if trace else None
    start_time = time.time()

    line_search_tool = get_line_search_tool(line_search_options)
    x_k = np.copy(x_0)
    alpha_k = None
    converged = False

    initial_grad_norm = np.linalg.norm(oracle.grad(x_k))
    
    if method == 'custom':
        x_k = np.zeros_like(x_0) # Начальное значение параметров w
        return x_k,  'success', 'not'

    for k in range(max_iter + 1):
        f_k = oracle.func(x_k)
        if np.isinf(f_k):
            return x_k, "computational_error", history
        grad_f_k = oracle.grad(x_k)
        grad_norm = np.linalg.norm(grad_f_k)
        if display:
            print("time: ", time.time() - start_time)
            print("func: ", f_k)
            print("grad_norm: ", grad_norm)
            print("x_k: ", x_k)
        if history is not None:
            history['time'].append(time.time() - start_time)
            history['func'].append(f_k)
            history['grad_norm'].append(grad_norm)
            if x_k.shape[0] < 3:
                history['x'].append(x_k)
        if grad_norm**2 <= tolerance * initial_grad_norm**2:
            converged = True
            break
        if k == max_iter:
            break
        d_k = (-1 * grad_f_k)
        alpha_k = line_search_tool.line_search(oracle, x_k, d_k, 2 * alpha_k if alpha_k is not None else None)
        x_k = x_k + alpha_k * d_k
#     if converged:
    return x_k, 'success', history
#     else:
#         return x_k, 'iterations_exceeded', history


def lossf(w, X, y, l1, l2):
    lossf = np.ones(y.shape).dot(np.logaddexp(np.zeros(y.shape), -1 * y * X.dot(w))) + \
                                                                      l1 * np.linalg.norm(w, ord=1) + \
                                                                      l2 * w.dot(w)
    return lossf


def gradf(w, X, y, l1, l2):
    gradient_cont_diff = -1.0 * X.T.dot(scipy.special.expit(-1 * y * X.dot(w)) * y) + 2 * l2 * w
    gradf = (gradient_cont_diff + l1 * np.sign(w)) * (np.abs(w) > 0) + \
            (w == 0) * ((gradient_cont_diff < -l1) * (gradient_cont_diff + l1) +
                        (gradient_cont_diff > l1) * (gradient_cont_diff - l1))
    return gradf


class LRL1Oracle:
    def __init__(self, X, y, l1=1e-4, l2=1e-4):
        self.X = X
        self.y = y
        self.l1 = l1
        self.l2 = l2

    def func(self, w):
        return lossf(w, self.X, self.y, self.l1, self.l2)

    def grad(self, w):
        return gradf(w, self.X, self.y, self.l1, self.l2)


class LR(object):
    def __init__(self, lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0):
        """
        Создание класса для лог регрессии
        :param lr: float, длина шага для оптимизатора
        :param l1: float, l1 коэффициент регуляризатора
        :param l2: float, l2 коэффициент регуляризатора
        :param num_iter: int, число итераций оптимизатора
        :param verbose: bool, ключик для вывода
        """
        self.l1 = l1
        self.l2 = l2
        self.w = None
        self.lr = lr
        self.verbose = verbose
        self.num_iter = num_iter

    def fit(self, X, y, method_='Constant', grad_method=None):
        """
        Обучение логистической регрессии.
        Настраивает self.w коэффициенты модели.
        Если self.verbose == True, то выводите значение
        функции потерь на итерациях метода оптимизации.
        :param X: numpy.array размера  (N, M), dtype = np.float
        :param y: numpy.array размера  (N,), dtype = np.int
        :return: self
        """

        oracle = LRL1Oracle(X, y, self.l1, self.l2)
        n, d = X.shape
        ls_tool = LineSearchTool(method=method_, c=self.lr)
        self.w = np.zeros(X.shape[1])
        res = gradient_descent(oracle, self.w, max_iter=self.num_iter,
                               line_search_options=ls_tool,
                               display=self.verbose, trace=True, tolerance=1e-5, method=grad_method)
        
#         w0, loss, info = optimize.fmin_l_bfgs_b(
#                     func, w0, fprime=None,
#                     args=(X, target, 1. / C, sample_weight),
#                     iprint=(verbose > 0) - 1, pgtol=tol, maxiter=max_iter)
        
#         print('okee', res[2])
        self.w = res[0]
        if res[1] != "success":
            raise Exception("fitting failed: " + res[1])
        return self

    def predict_proba(self, X):
        """
        Предсказание вероятности принадлежности объекта к классу 1.
        Возвращает np.array размера (N,) чисел в отрезке от 0 до 1.
        :param X: numpy.array размера  (N, M), dtype = np.float
        :return: numpy.array размера  (N,), dtype = np.float
        """
        # Вычислите вероятности принадлежности каждого
        # объекта из X к положительному классу, используйте
        # эту функцию для реализации LR.predict
        probs = scipy.special.expit(X.dot(self.w))
        return probs

    def predict(self, X):
        """
        Предсказание класса для объекта.
        Возвращает np.array размера (N,) элементов 1 или -1.
        :param X: numpy.array размера  (N, M), dtype = np.float
        :return:  numpy.array размера  (N,), dtype = np.int
        """
        # Вычислите предсказания для каждого объекта из X
        probs = self.predict_proba(X)
        predicts = np.ones(X.shape[0]) * (probs >= 0.5) - np.ones(X.shape[0]) * (probs < 0.5)
        return predicts
    
def _predict_proba_lr(X_t, w, intercept=0.52):
    """Probability estimation for OvR logistic regression.
    Positive class probabilities are computed as
    1. / (1. + np.exp(-self.decision_function(X)));
    multiclass is handled by normalizing that over all classes.
    """
    prob = X_t.dot(w) + intercept
    prob *= -1
    np.exp(prob, prob)
    prob += 1
    np.reciprocal(prob, prob)
    return np.vstack([1 - prob, prob]).T
    
if __name__ == '__main__':
    print("Hello World again from %s!" % __name__)


Hello World again from __main__!


In [799]:
def color_hist(img):    #bins_range=(0, 256)
    R = img[:,:,0].reshape(-1)
    G = img[:,:,1].reshape(-1)
    B = img[:,:,2].reshape(-1)
    responce = []
    for color in [R, G, B]:
        for threshold in [170, 210, 240]:
            responce.append(np.sum(color[color >= threshold]))
        for threshold in [80, 45, 10]:
            responce.append(np.sum(color[color <= threshold]))
    return responce

In [800]:
def get_stats_for_file(array_file, num_diff=3):
    responce = []
    array_file = np.array(array_file)
    for i in range(num_diff):
        if i == 0:
            responce += [array_file.mean(), array_file.min(), array_file.max(), array_file.std()]
        else:
            diff = np.diff(array_file , n = i)
            responce += [diff.mean(), diff.min(), diff.max(), diff.std()]
    return responce

In [801]:
def flip_vertical(img):
    return img[:, ::-1, :]

In [848]:
X = []
y = []
train_path = []

for img_path in tqdm(glob('./train/*.jpg')):
    img_orig = misc.imread(img_path)
    img_chromatic = misc.imread(img_path, mode='L')
    for img, img_crom in [(img_orig, img_chromatic), 
                                    (flip_vertical(img_orig), img_chromatic[:, ::-1])]:
        
        hg = hog_upd.hog(misc.imresize(img_crom, (64, 64)), flatten=True)
        hg_g = hog_upd.hog(misc.imresize(img[:, :, 1], (64, 64)), flatten=True)
        hg_b = hog_upd.hog(misc.imresize(img[:, :, 2], (64, 64)), flatten=True)
#         hg_r = hog_upd.hog(misc.imresize(img[:, :, 0], (64, 64)), flatten=True)
        
        hog = list(hg) + get_stats_for_file(hg, num_diff=2) + get_stats_for_file(hg_g, num_diff=3) + get_stats_for_file(hg_b, num_diff=3) + get_stats_for_file(hg_r, num_diff=3)
        img_col_hist = color_hist(img)
        img_col_hist += color_hist(img[:16, :, :])
        img_col_hist += color_hist(img[-16:, :, :])
        stats = get_stats_for_file(img[:, :, 2].reshape(-1))
        stats += get_stats_for_file(img[:, :, 1].reshape(-1))
#         stats += get_stats_for_file(img[:, :, 0].reshape(-1))             
        
        stats_half = []
        for sl in [1, 4, 8, 16]:
            for color in range(1, 3):
                stats_half += get_stats_for_file(img[:sl, :, color].reshape(-1))
                stats_half += get_stats_for_file(img[-sl:, :, color].reshape(-1))

        
        mean_row = list(np.mean(img[:, :, 2], axis=1))
        mean_row += list(np.mean(img[:, :, 1], axis=1))
        mean_row += list(np.mean(img[:, :, 0], axis=1))
        std_row = list(np.std(img[:, :, 2], axis=1))
        std_row += list(np.std(img[:, :, 1], axis=1))
        std_row += list(np.std(img[:, :, 0], axis=1))
        max_row = list(np.max(img[:, :, 2], axis=1))
        max_row += list(np.max(img[:, :, 1], axis=1))
        max_row += list(np.max(img[:, :, 0], axis=1))
        min_row = list(np.min(img[:, :, 2], axis=1))
        min_row += list(np.min(img[:, :, 1], axis=1))
        min_row += list(np.min(img[:, :, 0], axis=1))
        
        mean_col = list(np.mean(img[:, :, 2], axis=0))
        mean_col += list(np.mean(img[:, :, 1], axis=0))
        mean_col += list(np.mean(img[:, :, 0], axis=0))
        std_col = list(np.std(img[:, :, 2], axis=0))
        std_col += list(np.std(img[:, :, 1], axis=0))
        std_col += list(np.std(img[:, :, 0], axis=0))
        max_col = list(np.max(img[:, :, 2], axis=0))
        max_col += list(np.max(img[:, :, 1], axis=0))
        max_col += list(np.max(img[:, :, 0], axis=0))
        min_col = list(np.min(img[:, :, 2], axis=0))
        min_col += list(np.min(img[:, :, 1], axis=0))
        min_col += list(np.min(img[:, :, 0], axis=0))
        
        X.append(hog + img_col_hist + stats + stats_half + mean_row + std_row + max_row + min_row + mean_col + std_col + max_col + min_col)

        name = os.path.basename(img_path).split('_')[0].strip()
        class_ = 1 if name == 'indoor' else 0
        y.append(class_)
        train_path.append(img_path)

100%|██████████| 6905/6905 [03:04<00:00, 37.47it/s]


In [849]:
import gc

gc.collect()

4499

In [850]:
X = np.array(X)
y = np.array(y)
y_train = y[:5524 * 2]
y_test = y[5524 * 2:]
train_paths = train_path[:5524 * 2]
val_paths = train_path[5524 * 2:]
X_train = X[:5524 * 2, :]
X_test = X[5524 * 2:, :]
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [851]:
mean = np.mean(X_train, axis=0)
std = np.array(list(map(lambda st: st if st > 1e-15 else 1, np.std(X_train, axis=0))))

In [852]:
def scaler(X_last):
    X_scaled = (X_last - mean) / std
    return X_scaled

In [853]:
X_train = scaler(X_train)
X_test = scaler(X_test)

In [857]:
clf_1 = LR(num_iter=300, lr=0.1)
clf_1.fit(X_train, y_train, 'Wolfe')

In [861]:
def mse(y_pred, y_test):
    return np.sqrt( np.mean((y_test - y_pred) ** 2) )

In [873]:
y_pred = clf_1.predict_proba(X_test)
print(mse((y_pred[0::2] + y_pred[1::2]) / 2, y_test_01[0::2]))

0.209067248082


In [874]:
y_pred = _predict_proba_lr(X_test, clf_1.w)[:, 1]

In [875]:
print(mse((y_pred[0::2] + y_pred[1::2]) / 2, y_test_01[0::2]))

0.211387122524


In [465]:
X_test_kaggle = []
labels = []
for img_path in tqdm(glob('./test/*.jpg')):
    img_orig = misc.imread(img_path)
    img_chromatic = misc.imread(img_path, mode='L')
    for img, img_crom in [(img_orig, img_chromatic), (flip_vertical(img_orig), img_chromatic[:, ::-1])]:
        hg = hog_upd.hog(misc.imresize(img_crom, (64, 64)), flatten=True)
        hg_g = hog_upd.hog(misc.imresize(img[:, :, 1], (64, 64)), flatten=True)
        hg_b = hog_upd.hog(misc.imresize(img[:, :, 2], (64, 64)), flatten=True)
#         hg_r = hog_upd.hog(misc.imresize(img[:, :, 0], (64, 64)), flatten=True)
        
        hog = list(hg) + get_stats_for_file(hg, num_diff=2) + get_stats_for_file(hg_g, num_diff=3) + get_stats_for_file(hg_b, num_diff=3) + get_stats_for_file(hg_r, num_diff=3)
        img_col_hist = color_hist(img)
        img_col_hist += color_hist(img[:16, :, :])
        img_col_hist += color_hist(img[-16:, :, :])
        stats = get_stats_for_file(img[:, :, 2].reshape(-1))
        stats += get_stats_for_file(img[:, :, 1].reshape(-1))
#         stats += get_stats_for_file(img[:, :, 0].reshape(-1))             
        
        stats_half = []
        for sl in [1, 4, 8, 16]:
            for color in range(1, 3):
                stats_half += get_stats_for_file(img[:sl, :, color].reshape(-1))
                stats_half += get_stats_for_file(img[-sl:, :, color].reshape(-1))

        
        mean_row = list(np.mean(img[:, :, 2], axis=1))
        mean_row += list(np.mean(img[:, :, 1], axis=1))
        mean_row += list(np.mean(img[:, :, 0], axis=1))
        std_row = list(np.std(img[:, :, 2], axis=1))
        std_row += list(np.std(img[:, :, 1], axis=1))
        std_row += list(np.std(img[:, :, 0], axis=1))
        max_row = list(np.max(img[:, :, 2], axis=1))
        max_row += list(np.max(img[:, :, 1], axis=1))
        max_row += list(np.max(img[:, :, 0], axis=1))
        min_row = list(np.min(img[:, :, 2], axis=1))
        min_row += list(np.min(img[:, :, 1], axis=1))
        min_row += list(np.min(img[:, :, 0], axis=1))
        
        mean_col = list(np.mean(img[:, :, 2], axis=0))
        mean_col += list(np.mean(img[:, :, 1], axis=0))
        mean_col += list(np.mean(img[:, :, 0], axis=0))
        std_col = list(np.std(img[:, :, 2], axis=0))
        std_col += list(np.std(img[:, :, 1], axis=0))
        std_col += list(np.std(img[:, :, 0], axis=0))
        max_col = list(np.max(img[:, :, 2], axis=0))
        max_col += list(np.max(img[:, :, 1], axis=0))
        max_col += list(np.max(img[:, :, 0], axis=0))
        min_col = list(np.min(img[:, :, 2], axis=0))
        min_col += list(np.min(img[:, :, 1], axis=0))
        min_col += list(np.min(img[:, :, 0], axis=0))
#         X.append(hog + img_col_hist + list(misc.imresize(img_crom, (8, 8)).reshape(-1))  + stats + stats_half + mean_row + std_row + max_row + min_row +  mean_col + std_col + max_col + min_col)
        X_test_kaggle.append(hog + img_col_hist + stats + stats_half + mean_row + std_row + max_row + min_row + mean_col + std_col + max_col + min_col)
        
        name = int(os.path.basename(img_path).split('_')[-1].split('.')[0])
        labels.append(name)
    

100%|██████████| 2960/2960 [02:32<00:00, 19.32it/s]


In [466]:
X_test_kaggle = np.array(X_test_kaggle)
X_test_kaggle_scaled = scaler(X_test_kaggle)

In [ ]:
# не помню, как считал тут параметр, вроде одно из них, но скор и там, и там, хороший
y_pred = clf_1.predict_proba(X_test_kaggle_scaled)

In [469]:
y_pred = _predict_proba_lr(X_test_kaggle_scaled, clf_1.w)[:, 1]

In [485]:
y_pred = (y_pred[::2] + y_pred[1::2])/2

In [411]:
labels = np.array(labels)

In [413]:
labels = labels[::2]

## Засылаем

In [414]:
def write_answer(y_pred, labels):
    df = pd.concat([pd.DataFrame(labels), pd.DataFrame(y_pred)], axis=1)
    df.columns = ['id', 'res']
    df.index = df.id
    df = df.drop('id', axis=1)
    df.to_csv('hog_hist_visionhack_features_half.csv')

In [415]:
write_answer(y_pred, labels)